In [2]:
#import packages
import pandas as pd
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
stop_words = stopwords.words('english')
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])
import numpy as np
import gensim
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import time
label_encoder = preprocessing.LabelEncoder()

#____________________________________________________________________________________________________________________
#LOADING DATA + ORGANIZING DATASET (creating 'master_tweet' df for convenience, not used)
#load data downloded from the 'bot repository'
df_tfp = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\TFP.csv\tweets.csv")
df_fsf = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\FSF.csv\tweets.csv")
df_twt = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\TWT.csv\tweets.csv")
df_int = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\INT.csv\tweets.csv")
df_e13 = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\E13.csv\tweets.csv")


df_tfp_u = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\TFP.csv\users.csv")
df_fsf_u = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\FSF.csv\users.csv")
df_twt_u = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\TWT.csv\users.csv")
df_int_u = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\INT.csv\users.csv")
df_e13_u = pd.read_csv(r"C:\Users\wac6er\Downloads\Fake_project_dataset_csv.tar\Fake_project_dataset_csv\E13.csv\users.csv")

#aggregating user tweet + bio text data
pieces = (df_tfp,df_fsf,df_twt,df_int,df_e13)
df_master_tweet = pd.concat(pieces, ignore_index = True)
df_master_tweet['check'] = df_master_tweet['user_id']
df_master_tweet['new'] = df_master_tweet['text'].str.split().str.get(0)
df_master_tweet = df_master_tweet[df_master_tweet.new != 'RT']

pieces = (df_tfp_u,df_fsf_u,df_twt_u,df_int_u,df_e13_u)
df_master_user = pd.concat(pieces, ignore_index = True)
df_master_user['check'] = df_master_user['user_id']
df_master_user['new'] = df_master_user['text'].str.split().str.get(0)
df_master_user = df_master_user[df_master_user.new != 'RT']

#merging user tweets + bios
df_all = pd.merge(df_master_tweet,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all = df_all.drop_duplicates()

#printing length of the 'true user' dataset, then sampling that number from the different bot dataset tweets
print(len(df_all[df_all['dataset']=='TFP']))

number_real =89
number_fake = 25 #roughly 1/4

tfp_df = df_all[df_all['dataset']=='TFP'].sample(n=number_real)
int_df = df_all[df_all['dataset']=='INT'].sample(n=number_fake)
e13_df = df_all[df_all['dataset']=='E13'].sample(n=number_fake)
fsf_df = df_all[df_all['dataset']=='FSF'].sample(n=number_fake)
twt_df = df_all[df_all['dataset']=='TWT'].sample(n=number_fake)
pcs = (tfp_df,int_df,e13_df,fsf_df,twt_df)

#combining sampled dataset
df_all = pd.concat(pcs, ignore_index = True)

#____________________________________________________________________________________________________________________
#CREATING SCATTER MATRIX + CLEANING
min_chars = 3
max_feats = 70000
min_freq = 15

#removing all non-alphanumeric numbers
df_all['tweet_re'] = df_all.text.str.replace('[^a-zA-Z]', ' ')
df_all['tweet_re'] = df_all['text'].map(lambda x:x.lower())

#replacing meaningless characters
df_all = df_all.replace({'object':' '}, regex = True)
df_all = df_all.replace({'int':' '}, regex = True)
df_all = df_all.replace({'http':' '}, regex = True)
df_all = df_all.replace({'dtype':' '}, regex = True)

#replacing all stopwords
df_all['tweet_no_stop'] = df_all['tweet_re'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

#tokenizing results
df_all['tweet_token'] = df_all['tweet_no_stop'].apply(lambda x: x.split(" "))

#lemmatizing words to extract roots (distinguishing fished/fishing/fish for ex)
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(df_all['tweet_token'], allowed_postags=['NOUN', 'VERB'])

#creating document weight vector
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_freq,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=max_feats)

data_vectorized = vectorizer.fit_transform(data_lemmatized)

c#RUNNING THE LDA MODEL, attempting 8 different topic numbers
xxx=[2,3,4,5,6,7,8,9]
chart = {'perp':[],'score':[]}
for i in xxx:
    lda_model = LatentDirichletAllocation(n_components=i,         
                      max_iter=20,               
                      learning_method='online',   
                      random_state=100,          
                      batch_size=128,            
                      evaluate_every = -1,       
                      n_jobs = -1)

    lda_output = lda_model.fit_transform(data_vectorized)

    chart['score'].append(lda_model.score(data_vectorized))
    chart['perp'].append(lda_model.perplexity(data_vectorized))

#chart contains accuracy variables
chart = pd.DataFrame(chart)

#checking out topics + their word weight
def selected_topics(model, vectorizer, top_n=20):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

selected_topics(lda_model, vectorizer)


#loading LDA outputs (per tweet, % assignment to each topic) to a csv to merge with numeric tweet data such as likes/comments/followers/etc
lda_model = LatentDirichletAllocation(n_components=2,         
                  max_iter=20,               
                  learning_method='online',   
                  random_state=100,          
                  batch_size=128,            
                  evaluate_every = -1,       
                  n_jobs = -1)

lda_output2 = lda_model.fit_transform(data_vectorized)

lda_model = LatentDirichletAllocation(n_components=3,         
                  max_iter=20,               
                  learning_method='online',   
                  random_state=100,          
                  batch_size=128,            
                  evaluate_every = -1,       
                  n_jobs = -1)

lda_output3 = lda_model.fit_transform(data_vectorized)

lda_model = LatentDirichletAllocation(n_components=4,         
                  max_iter=20,               
                  learning_method='online',   
                  random_state=100,          
                  batch_size=128,            
                  evaluate_every = -1,       
                  n_jobs = -1)

lda_output4 = lda_model.fit_transform(data_vectorized)

lda_model = LatentDirichletAllocation(n_components=5,         
                  max_iter=20,               
                  learning_method='online',   
                  random_state=100,          
                  batch_size=128,            
                  evaluate_every = -1,       
                  n_jobs = -1)

lda_output5 = lda_model.fit_transform(data_vectorized)

lda_model = LatentDirichletAllocation(n_components=6,         
                  max_iter=20,               
                  learning_method='online',   
                  random_state=100,          
                  batch_size=128,            
                  evaluate_every = -1,       
                  n_jobs = -1)

lda_output6 = lda_model.fit_transform(data_vectorized)

lda_model = LatentDirichletAllocation(n_components=7,         
                  max_iter=20,               
                  learning_method='online',   
                  random_state=100,          
                  batch_size=128,            
                  evaluate_every = -1,       
                  n_jobs = -1)

lda_output7 = lda_model.fit_transform(data_vectorized)

lda_model = LatentDirichletAllocation(n_components=8,         
                  max_iter=20,               
                  learning_method='online',   
                  random_state=100,          
                  batch_size=128,            
                  evaluate_every = -1,       
                  n_jobs = -1)

lda_output8 = lda_model.fit_transform(data_vectorized)

df2 = pd.DataFrame(lda_output2)
df3 = pd.DataFrame(lda_output3)
df4 = pd.DataFrame(lda_output4)
df5 = pd.DataFrame(lda_output5)
df6 = pd.DataFrame(lda_output6)
df7 = pd.DataFrame(lda_output7)
df8 = pd.DataFrame(lda_output8)

#saving
df2.to_csv('lda2.csv')
df3.to_csv('lda3.csv')
df4.to_csv('lda4.csv')
df5.to_csv('lda5.csv')
df6.to_csv('lda6.csv')
df7.to_csv('lda7.csv')
df8.to_csv('lda8.csv')

#____________________________________________________________________________________________________________________
#USING LDA RESULTS + INTITIAL DATASET WITH NUMERIC ATTRIBUTES TO RUN A RANDOM FOREST
#reading in data with LDA results (MANNUALLY MERGED DATASETS GIVEN LACK OF COMMON IDENTIFIER TO MERGE ON) 
lda2 = pd.read_csv('/Users/willcarrington/Downloads/lda2_final.csv')
lda3 = pd.read_csv('/Users/willcarrington/Downloads/lda3_final.csv')
lda4 = pd.read_csv('/Users/willcarrington/Downloads/lda4_final.csv')
lda5 = pd.read_csv('/Users/willcarrington/Downloads/lda5_final.csv')
lda6 = pd.read_csv('/Users/willcarrington/Downloads/lda6_final.csv')
lda7 = pd.read_csv('/Users/willcarrington/Downloads/lda7_final.csv')
lda8 = pd.read_csv('/Users/willcarrington/Downloads/lda8_final.csv')

#creating dummy variables 
df_new = lda2
df_new[['a','b','c','label','e']] = pd.get_dummies(lda5['dataset'])
df_new['dataset']= label_encoder.fit_transform(df_new['dataset']) 


#running a random forest with + without LDA results to check for improvement
xx = df_new[['0','1','statuses_count','followers_count','friends_count','listed_count']]
xx=xx.fillna(0)

y = df_new['label']
x = xx
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf=RandomForestClassifier(n_estimators=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

df_new = lda3
df_new[['a','b','c','label','e']] = pd.get_dummies(lda5['dataset'])
df_new['dataset']= label_encoder.fit_transform(df_new['dataset']) 


xx = df_new[['0','1','2','statuses_count','followers_count','friends_count','listed_count']]
xx=xx.fillna(0)

y = df_new['label']
x = xx
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf=RandomForestClassifier(n_estimators=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

df_new = lda4
df_new[['a','b','c','label','e']] = pd.get_dummies(lda5['dataset'])
df_new['dataset']= label_encoder.fit_transform(df_new['dataset']) 


xx = df_new[['0','1','2','3','statuses_count','followers_count','friends_count','listed_count']]
xx=xx.fillna(0)

y = df_new['label']
x = xx
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf=RandomForestClassifier(n_estimators=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

df_new = lda5
df_new[['a','b','c','label','e']] = pd.get_dummies(lda5['dataset'])
df_new['dataset']= label_encoder.fit_transform(df_new['dataset']) 


xx = df_new[['0','1','2','3','4','statuses_count','followers_count','friends_count','listed_count']]
xx=xx.fillna(0)

y = df_new['label']
x = xx
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf=RandomForestClassifier(n_estimators=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

df_new = lda6
df_new[['a','b','c','label','e']] = pd.get_dummies(lda5['dataset'])
df_new['dataset']= label_encoder.fit_transform(df_new['dataset']) 


xx = df_new[['0','1','2','3','4','5','statuses_count','followers_count','friends_count','listed_count']]
xx=xx.fillna(0)

y = df_new['label']
x = xx
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf=RandomForestClassifier(n_estimators=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

df_new = lda7
df_new[['a','b','c','label','e']] = pd.get_dummies(lda5['dataset'])
df_new['dataset']= label_encoder.fit_transform(df_new['dataset']) 


xx = df_new[['0','1','2','3','4','5','6','statuses_count','followers_count','friends_count','listed_count']]
xx=xx.fillna(0)

y = df_new['label']
x = xx
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf=RandomForestClassifier(n_estimators=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

df_new = lda8
df_new[['a','b','c','label','e']] = pd.get_dummies(lda5['dataset'])
df_new['dataset']= label_encoder.fit_transform(df_new['dataset']) 


xx = df_new[['0','1','2','3','4','5','6','7','statuses_count','followers_count','friends_count','listed_count']]
xx=xx.fillna(0)

y = df_new['label']
x = xx
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf=RandomForestClassifier(n_estimators=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))
#____________________________________________________________________________________________________________________
#SCRAP YARD
df_master_tweet = pd.read_csv('/Users/willcarrington/Downloads/master_tweet.csv')
df_master_user = pd.read_csv('/Users/willcarrington/Downloads/master_user.csv')

i = 140000
num = 140000*15

df = df.sample(n=num)

df_master_tweet = df.iloc[:140000]
df_master_tweet1= df.iloc[140000:280000]
df_master_tweet2= df.iloc[280000:420000]
df_master_tweet3= df.iloc[420000:560000]
df_master_tweet4= df.iloc[560000:700000]
df_master_tweet5= df.iloc[700000:840000]
df_master_tweet6= df.iloc[840000:980000]
df_master_tweet7= df.iloc[980000:980000+i]
df_master_tweet8= df.iloc[980000+i:980000+i+i]
df_master_tweet9= df.iloc[980000+i+i:980000+i+i+i]
df_master_tweet10= df.iloc[980000+i+i+i:980000+i+i+i+i]
df_master_tweet11= df.iloc[980000+i+i+i+i:980000+i+i+i+i+i]
df_master_tweet12= df.iloc[980000+i+i+i+i+i:980000+i+i+i+i+i+i]
df_master_tweet13= df.iloc[980000+i+i+i+i+i+i:980000+i+i+i+i+i+i+i]
df_master_tweet14= df.iloc[980000+i+i+i+i+i+i+i:980000+i+i+i+i+i+i+i+i]
df_master_tweet15= df.iloc[980000+i+i+i+i+i+i+i+i:980000+i+i+i+i+i+i+i+i+i]

df_all = pd.merge(df_master_tweet,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet1,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all1 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet2,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all2 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet3,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all3 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet4,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all4 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet5,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all5 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet6,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all6 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet7,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all7 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet8,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all8 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet9,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all9 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet10,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all10 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet11,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all11 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet12,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all12 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet13,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all13 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet14,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all14 = df_all.drop_duplicates()

df_all = pd.merge(df_master_tweet15,df_master_user,on='check')
df_all = df_all[df_all['lang']=='en']
df_all = df_all[['text','check','dataset','statuses_count','followers_count','friends_count','listed_count']]
df_all['text'] = df_all.groupby(['check'])['text'].transform(lambda x : ''.join(str(x)))
df_all15 = df_all.drop_duplicates()

pieces = (df_all1,df_all2,df_all3,df_all4,df_all5,df_all6,df_all7,df_all8,df_all9,df_all10,df_all11,df_all12,df_all13,df_all14,df_all15)
df_master_split = pd.concat(pieces, ignore_index = True)
df_master_split.head(500000)

print(len(df_master_split[df_master_split['dataset']=='TFP']))
print(len(df_master_split[df_master_split['dataset']=='FSF']))
print(len(df_master_split[df_master_split['dataset']=='E13']))
print(len(df_master_split[df_master_split['dataset']=='INT']))
print(len(df_master_split[df_master_split['dataset']=='TWT']))

number_real =1138
number_fake = 284

tfp_df = df_master_split[df_master_split['dataset']=='TFP'].sample(n=number_real)
int_df = df_master_split[df_master_split['dataset']=='INT'].sample(n=number_fake)
e13_df = df_master_split[df_master_split['dataset']=='E13'].sample(n=number_fake)
fsf_df = df_master_split[df_master_split['dataset']=='FSF'].sample(n=number_fake)
twt_df = df_master_split[df_master_split['dataset']=='TWT'].sample(n=number_fake)
pcs = (tfp_df,int_df,e13_df,fsf_df,twt_df)
df_master_split = pd.concat(pcs, ignore_index = True)

min_chars = 4
max_feats = 70000
min_freq = 15


df_master_split['tweet_re'] = df_master_split.text.str.replace('[^a-zA-Z]', ' ')
df_master_split['tweet_re'] = df_master_split['text'].map(lambda x:x.lower())

df_master_split = df_master_split.replace({'object':' '}, regex = True)
df_master_split = df_master_split.replace({'int':' '}, regex = True)
df_master_split = df_master_split.replace({'http':' '}, regex = True)
df_master_split = df_master_split.replace({'dtype':' '}, regex = True)


df_master_split['tweet_no_stop'] = df_master_split['tweet_re'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

df_master_split['tweet_token'] = df_master_split['tweet_no_stop'].apply(lambda x: x.split(" "))


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(df_master_split['tweet_token'], allowed_postags=['NOUN', 'VERB'])

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_freq,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=max_feats)

data_vectorized = vectorizer.fit_transform(data_lemmatized)

xxx=[2,3,4,5,6,7,8,9]
chart = {'perp':[],'score':[]}
for i in xxx:
    lda_model = LatentDirichletAllocation(n_components=i,         
                      max_iter=20,               
                      learning_method='online',   
                      random_state=100,          
                      batch_size=128,            
                      evaluate_every = -1,       
                      n_jobs = -1)

    lda_output = lda_model.fit_transform(data_vectorized)

    chart['score'].append(lda_model.score(data_vectorized))
    chart['perp'].append(lda_model.perplexity(data_vectorized))
chart = pd.DataFrame(chart)

start_time = time.time()
importances = clf.feature_importances_
predictions = clf.predict_proba(X_train)
std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.9f} seconds")
Out

import matplotlib.pyplot as plt 
forest_importances = pd.Series(importances,index = range(8))

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/willcarrington/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
//anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
//anaconda3/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
//anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
//anaconda3/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
//anaconda3/lib/python3.7/sit